In [ ]:
!pip install "cdsapi>=0.7.4"
!pip install netCDF4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 43.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import cdsapi

In [ ]:
#!rm -rf /content/IGNIS

In [ ]:
!git clone https://github.com/FaridRash/IGNIS

Cloning into 'IGNIS'...
remote: Enumerating objects: 340, done.
remote: Counting objects: 100% (136/136), done.
remote: Compressing objects: 100% (116/116), done.
remote: Total 340 (delta 68), reused 61 (delta 20), pack-reused 204 (from 1)
Receiving objects: 100% (340/340), 44.33 MiB | 13.80 MiB/s, done.
Resolving deltas: 100% (148/148), done.


In [ ]:
!git clone https://github.com/FaridRash/IGNIS IGNIS
!rm -rf /content/IGNIS/Dataset
!rm -rf /content/IGNIS/FoV\ visualization
!rm -rf /content/IGNIS/Notebooks
!rm -rf /content/IGNIS/Reports
!rm -rf /content/IGNIS/Requests
!rm -rf /content/IGNIS/SNR\ Visualization



fatal: destination path 'IGNIS' already exists and is not an empty directory.


In [ ]:
df_excel = pd.read_csv('/content/IGNIS/Location/df_excel_with_temperature.csv')

print(df_excel.head())

     Altitude      Theta   Hour  Month  Day        GSD      North      South  \
0  461.218874  21.299047  02:00      1    2  76.025089  40.829036  40.828186   
1  462.767619  12.416525  02:00      1    5  76.280377  40.829036  40.828186   
2  463.422734  14.037020  02:00      1    5  76.388363  40.829036  40.828186   
3  466.582339  22.475595  15:00      1    5  76.909177  40.829036  40.828186   
4  466.012605  13.759815  15:00      1    5  76.815265  40.829036  40.828186   

        West       East  Background Temperature  
0  14.144160  14.145534              286.143799  
1  14.144157  14.145531              287.206543  
2  14.144156  14.145530              287.206543  
3  14.144150  14.145524              288.963867  
4  14.144151  14.145525              288.963867  


In [ ]:
cdsapirc = """
url: https://cds.climate.copernicus.eu/api
key: d97ec102-82f7-4827-8b82-13721a50119b

"""

with open('/root/.cdsapirc', 'w') as f:
    f.write(cdsapirc)

In [ ]:
import cdsapi
import xarray as xr
import pandas as pd
import time

for index, row in df_excel.iterrows():
    month = f"{int(row['Month']):02d}"
    day = f"{int(row['Day']):02d}"
    hour = row['Hour']
    north = row['North']
    south = row['South']
    west = row['West']
    east = row['East']

    print(f"\n📦 Requesting data for Row {index}")
    print(f"Date: 2024-{month}-{day}, Hour: {hour}")
    print(f"Bounding Box: N={north}, S={south}, W={west}, E={east}")
    c = cdsapi.Client()
    c.retrieve(
        'reanalysis-era5-single-levels',
        {
            'product_type': 'reanalysis',
            'format': 'netcdf',
            'variable': ['2m_temperature'],
            'year': '2024',
            'month': month,
            'day': day,
            'time': hour,
            'area': [north, west, south, east],
        },
        'download.nc'
    )

    try:
        ds = xr.open_dataset('download.nc')
        print("📖 Available dimensions:", ds.dims)
        print("📖 Available variables:", list(ds.data_vars))

        # Use available dimensions safely
        if 'valid_time' in ds.dims:
            temp = ds['t2m'].isel(valid_time=0).mean().values.item()
        else:
            temp = ds['t2m'].isel(time=0).mean().values.item()

        df_excel.at[index, 'Background Temperature'] = temp
        print(f"✅ Extracted Temperature: {temp} K")

    except Exception as e:
        print(f"❌ Error reading NetCDF for row {index}: {e}")
        df_excel.at[index, 'Background Temperature'] = None

    time.sleep(1)



📦 Requesting data for Row 0
Date: 2024-01-02, Hour: 02:00
Bounding Box: N=40.829036, S=40.828186, W=14.144160436041204, E=14.145534436041205

📦 Requesting data for Row 1
Date: 2024-01-05, Hour: 02:00
Bounding Box: N=40.829036, S=40.828186, W=14.14415740528726, E=14.145531405287262

📦 Requesting data for Row 2
Date: 2024-01-05, Hour: 02:00
Bounding Box: N=40.829036, S=40.828186, W=14.14415612328632, E=14.14553012328632

📦 Requesting data for Row 3
Date: 2024-01-05, Hour: 15:00
Bounding Box: N=40.829036, S=40.828186, W=14.144149940220998, E=14.145523940221

📦 Requesting data for Row 4
Date: 2024-01-05, Hour: 15:00
Bounding Box: N=40.829036, S=40.828186, W=14.144151055140505, E=14.145525055140505

📦 Requesting data for Row 5
Date: 2024-01-08, Hour: 15:00
Bounding Box: N=40.829036, S=40.828186, W=14.144153621704175, E=14.145527621704176

📦 Requesting data for Row 6
Date: 2024-01-13, Hour: 02:00
Bounding Box: N=40.829036, S=40.828186, W=14.144150452160575, E=14.145524452160576

📦 Requestin

2025-06-10 06:56:21,274 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
INFO:ecmwf.datastores.legacy_client:[2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-06-10 06:56:22,315 INFO Request ID is 67bc33f1-b675-4656-b120-405e9800682a
INFO:ecmwf.datastores.legacy_client:Request ID is 67bc33f1-b675-4656-b120-405e9800682a
2025-06-10 06:56:22,536 INFO status has been updated to accepted
INFO:ecmwf.datastores.legacy_client:status has been updated to accepted
2025-06-10 06:56:45,010 INFO status has been updated to successful
INFO:ecmwf.datastores.legacy_client:status has been updated to successful


c7ceca09062219d5da29d252bf10b474.nc:   0%|          | 0.00/25.0k [00:00<?, ?B/s]

📖 Available dimensions: FrozenMappingWarningOnValuesAccess({'valid_time': 1, 'latitude': 1, 'longitude': 1})
📖 Available variables: ['t2m']
✅ Extracted Temperature: 292.9873046875 K


In [ ]:
df_excel[:1126]

,Altitude,Theta,Hour,Month,Day,GSD,North,South,West,East,Background Temperature
0,461.218874,21.299047,02:00,1,2,76.025089,40.829036,40.828186,14.144160,14.145534,286.143799
1,462.767619,12.416525,02:00,1,5,76.280377,40.829036,40.828186,14.144157,14.145531,287.206543
2,463.422734,14.037020,02:00,1,5,76.388363,40.829036,40.828186,14.144156,14.145530,287.206543
3,466.582339,22.475595,15:00,1,5,76.909177,40.829036,40.828186,14.144150,14.145524,288.963867
4,466.012605,13.759815,15:00,1,5,76.815265,40.829036,40.828186,14.144151,14.145525,288.963867
...,...,...,...,...,...,...,...,...,...,...,...
1121,239.493478,5.665667,03:00,9,18,39.476947,40.829036,40.828186,14.144594,14.145968,292.074219
1122,217.520348,31.291111,16:00,9,21,35.855002,40.829036,40.828186,14.144637,14.146011,295.946045
1123,216.994809,38.488131,16:00,9,21,35.768375,40.829036,40.828186,14.144638,14.146012,295.946045
1124,200.498989,30.063427,03:00,9,23,33.049284,40.829036,40.828186,14.144671,14.146045,292.987305


In [ ]:
df_excel.to_csv('df_excel_with_temperature.csv', index=False)

In [ ]:
from pathlib import Path

token = "Token"
repo = "FaridRash/IGNIS"
repo_url = f"https://{token}@github.com/{repo}.git"

# Configure Git
!git config --global user.email "farid.rash@gmail.com"
!git config --global user.name "FaridRash"

# Save your CSV again to be sure it's there
!cp df_excel_with_temperature.csv /content/IGNIS/

# Go to your repo
%cd /content/IGNIS

# Add, commit, and push
!git add df_excel_with_temperature.csv
!git commit -m "Add df_excel_with_temperature.csv to repo root"
!git push {repo_url} main


/content/IGNIS
[main 27ea8f8] Add df_excel_with_temperature.csv to repo root
 2 files changed, 1128 insertions(+), 1 deletion(-)
 create mode 100644 df_excel_with_temperature.csv
Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 2 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 450 bytes | 450.00 KiB/s, done.
Total 4 (delta 3), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
To https://github.com/FaridRash/IGNIS.git
   ea2c2ed..27ea8f8  main -> main
